# Import Libraries

In [14]:
import pickle
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import sys
sys.path.append('/home/adedapo/personal_project/daps05ayoade/DeepTrade')

from secrecy import api_key
from ml_logic.scraping_stock_data import get_stock_data
from ml_logic.preprocessing import window

2023-11-07 14:23:38.297868: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 14:23:38.372872: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 14:23:38.798115: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 14:23:38.799519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 14:23:40.283166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [8]:
# Get the stock data
ticker = 'INTC'
df = get_stock_data(ticker, api_key)
df.head(2)

,open,high,low,close,adj_close,volume,dividend,split_coeff,ema,macd,...,upper_band,middle_band,lower_band,rsi,slowk,slowd,atr,returns,rolling_std,target
1999-12-17,81.38,83.00,80.50,82.06,23.620510,34734800,0.0,1.0,22.3326,-0.0086,...,23.8450,21.9789,20.1129,63.2875,93.5911,75.0440,0.9125,0.022555,0.037608,23.332665
1999-12-20,82.88,83.19,79.13,81.06,23.332665,18125700,0.0,1.0,22.5145,0.0708,...,24.0965,22.0705,20.0444,60.0158,88.2866,84.6461,0.9381,-0.012186,0.037697,23.856542


In [6]:
# Preprocess the data
X_train, y_train, X_val, y_val, X_test, y_test, sc = window(df)

print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}\nX_val: {X_val.shape}\ny_val: {y_val.shape}\nX_test shape: {X_test.shape}\ny_test shape: {y_test.shape}')

X_train shape: (4146, 60, 21)
y_train shape: (4146,)
X_val: (841, 60, 21)
y_val: (841,)
X_test shape: (842, 60, 21)
y_test shape: (842,)


In [12]:
batch_size=15
epochs=1000

# Save the scaler for the ticker
with open(f'/home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_scaler/scaler_{ticker}.pkl', 'wb') as file:
    pickle.dump(sc, file)

# Model: RNN

In [15]:
# Define the RNN model
rnn = Sequential()

# Add the first SimpleRNN layer
rnn.add(SimpleRNN(units=30, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))

# Add a second SimpleRNN layer
rnn.add(SimpleRNN(units=30, return_sequences=False))

# Add a Dense layer
rnn.add(Dense(units=25))

# Output layer
rnn.add(Dense(units=1))

# Compile model
rnn.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Checkpoint callback to save best model during training
checkpoint_filepath = f'/home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_model/best_rnn_model_{ticker}.h5'
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_mae', save_best_only=True, verbose=1)

# Early stopping callback to stop training when Mean Absolute Error stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1)

# Fit and train model
rnn.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[checkpoint, early_stopping])

Epoch 1/1000
276/277 [============================>.] - ETA: 0s - loss: 42.8838 - mae: 4.0061
Epoch 1: val_mae improved from inf to 11.75361, saving model to /home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_model/best_rnn_model_INTC.h5
277/277 [==============================] - 5s 15ms/step - loss: 42.8223 - mae: 4.0010 - val_loss: 193.3765 - val_mae: 11.7536
Epoch 2/1000
  9/277 [..............................] - ETA: 3s - loss: 1.0836 - mae: 0.6182

/home/adedapo/.pyenv/versions/3.10.6/envs/Lewagon/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


277/277 [==============================] - ETA: 0s - loss: 1.4344 - mae: 0.6588
Epoch 2: val_mae improved from 11.75361 to 5.80140, saving model to /home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_model/best_rnn_model_INTC.h5
277/277 [==============================] - 4s 14ms/step - loss: 1.4344 - mae: 0.6588 - val_loss: 60.6305 - val_mae: 5.8014
Epoch 3/1000
274/277 [============================>.] - ETA: 0s - loss: 0.6476 - mae: 0.5263
Epoch 3: val_mae improved from 5.80140 to 4.53453, saving model to /home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_model/best_rnn_model_INTC.h5
277/277 [==============================] - 4s 14ms/step - loss: 0.6482 - mae: 0.5275 - val_loss: 42.4874 - val_mae: 4.5345
Epoch 4/1000
277/277 [==============================] - ETA: 0s - loss: 0.5995 - mae: 0.5255
Epoch 4: val_mae improved from 4.53453 to 4.13504, saving model to /home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_model/best_rnn_model_INTC.h5
277/277 [=============

277/277 [==============================] - 4s 16ms/step - loss: 0.4612 - mae: 0.4699 - val_loss: 15.7931 - val_mae: 2.4612
Epoch 27/1000
274/277 [============================>.] - ETA: 0s - loss: 0.4670 - mae: 0.4729
Epoch 27: val_mae did not improve from 2.46122
277/277 [==============================] - 4s 15ms/step - loss: 0.4696 - mae: 0.4734 - val_loss: 16.8323 - val_mae: 2.5298
Epoch 28/1000
275/277 [============================>.] - ETA: 0s - loss: 0.4275 - mae: 0.4529
Epoch 28: val_mae improved from 2.46122 to 2.44463, saving model to /home/adedapo/personal_project/daps05ayoade/DeepTrade/temp_model/best_rnn_model_INTC.h5
277/277 [==============================] - 4s 15ms/step - loss: 0.4288 - mae: 0.4526 - val_loss: 15.3401 - val_mae: 2.4446
Epoch 29/1000
274/277 [============================>.] - ETA: 0s - loss: 0.4392 - mae: 0.4514
Epoch 29: val_mae did not improve from 2.44463
277/277 [==============================] - 4s 14ms/step - loss: 0.4374 - mae: 0.4507 - val_loss: 18

275/277 [============================>.] - ETA: 0s - loss: 0.3839 - mae: 0.4304
Epoch 56: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3831 - mae: 0.4300 - val_loss: 18.0580 - val_mae: 2.7005
Epoch 57/1000
275/277 [============================>.] - ETA: 0s - loss: 0.4000 - mae: 0.4402
Epoch 57: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3992 - mae: 0.4398 - val_loss: 15.4077 - val_mae: 2.4540
Epoch 58/1000
277/277 [==============================] - ETA: 0s - loss: 0.3825 - mae: 0.4267
Epoch 58: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3825 - mae: 0.4267 - val_loss: 17.0225 - val_mae: 2.6822
Epoch 59/1000
274/277 [============================>.] - ETA: 0s - loss: 0.4104 - mae: 0.4425
Epoch 59: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.4096 - mae: 0.4

276/277 [============================>.] - ETA: 0s - loss: 0.3228 - mae: 0.4049
Epoch 87: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3225 - mae: 0.4048 - val_loss: 25.6722 - val_mae: 3.1349
Epoch 88/1000
275/277 [============================>.] - ETA: 0s - loss: 0.3152 - mae: 0.3997
Epoch 88: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3152 - mae: 0.3997 - val_loss: 26.8781 - val_mae: 3.3471
Epoch 89/1000
274/277 [============================>.] - ETA: 0s - loss: 0.3339 - mae: 0.4182
Epoch 89: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3342 - mae: 0.4188 - val_loss: 26.5214 - val_mae: 3.2561
Epoch 90/1000
276/277 [============================>.] - ETA: 0s - loss: 0.3293 - mae: 0.4058
Epoch 90: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.3289 - mae: 0.4

Epoch 118/1000
274/277 [============================>.] - ETA: 0s - loss: 0.2938 - mae: 0.3871
Epoch 118: val_mae did not improve from 2.20951
277/277 [==============================] - 5s 16ms/step - loss: 0.2934 - mae: 0.3868 - val_loss: 42.6572 - val_mae: 4.3146
Epoch 119/1000
276/277 [============================>.] - ETA: 0s - loss: 0.2851 - mae: 0.3851
Epoch 119: val_mae did not improve from 2.20951
277/277 [==============================] - 5s 16ms/step - loss: 0.2847 - mae: 0.3847 - val_loss: 40.6590 - val_mae: 4.1577
Epoch 120/1000
277/277 [==============================] - ETA: 0s - loss: 0.2745 - mae: 0.3809
Epoch 120: val_mae did not improve from 2.20951
277/277 [==============================] - 4s 16ms/step - loss: 0.2745 - mae: 0.3809 - val_loss: 37.4622 - val_mae: 3.9693
Epoch 121/1000
274/277 [============================>.] - ETA: 0s - loss: 0.2927 - mae: 0.3955
Epoch 121: val_mae did not improve from 2.20951
277/277 [==============================] - 5s 16ms/step - l